# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments  \
0  Sep 26 2022 10:50AM  247255        10  8559096-SD.1   
1  Sep 26 2022 10:46AM  247246        21        627955   
2  Sep 26 2022 10:41AM  247253        15       8574214   
3  Sep 26 2022 10:41AM  247253        15       8574215   
4  Sep 26 2022 10:41AM  247253        15       8574216   
5  Sep 26 2022 10:41AM  247253        15       8574817   
6  Sep 26 2022 10:41AM  247253        15       8575338   
7  Sep 26 2022 10:41AM  247253        15       8575339   
8  Sep 26 2022 10:40AM  247249        15       8560864   
9  Sep 26 2022 10:40AM  247249        15       8574244   

                                Customer ShipmentStatus  
0                       Eywa Pharma Inc.       Released  
1                      NBTY Global, Inc.       Released  
2  Carwin Pharmaceutical Associates, LLC       Released  
3  Carwin Pharmaceutical Associates, LLC       Released  
4  Carwin Pharmaceutical Associates, LLC       Released  
5  Carwin Pharmaceutical Associates, LLC       Released  
6  Carwin Pharmaceutical Associates, LLC       Released  
7  Carwin Pharmaceutical Associates, LLC       Released  
8        Brookfield Pharmaceuticals, LLC       Released  
9        Brookfield Pharmaceuticals, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
20  247248       Released          8
21  247249       Released         24
22  247252       Released          5
23  247253       Released          6
24  247255       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
247248                NaN       8.0
247249                NaN      24.0
247252                NaN       5.0
247253                NaN       6.0
247255                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
247197                1.0       0.0
247199                1.0       0.0
247215                0.0       1.0
247217                0.0       1.0
247219                0.0      15.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
247197                  1         0
247199                  1         0
247215                  0         1
247217                  0         1
247219                  0        15

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               247197          1         0
1               247199          1         0
2               247215          0         1
3               247217          0         1
4               247219          0        15

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               247197         1         
1               247199         1         
2               247215                  1
3               247217                  1
4               247219                 15

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0    Sep 26 2022 10:50AM  247255        10   
1    Sep 26 2022 10:46AM  247246        21   
2    Sep 26 2022 10:41AM  247253        15   
8    Sep 26 2022 10:40AM  247249        15   
32   Sep 26 2022 10:39AM  247248        15   
40   Sep 26 2022 10:29AM  247252        20   
45   Sep 26 2022 10:05AM  247247        12   
54   Sep 26 2022 10:03AM  247229        10   
99   Sep 26 2022  9:56AM  247199        21   
100  Sep 26 2022  9:54AM  247242        10   

                                  Customer  
0                         Eywa Pharma Inc.  
1                        NBTY Global, Inc.  
2    Carwin Pharmaceutical Associates, LLC  
8          Brookfield Pharmaceuticals, LLC  
32                   Alliance Pharma, Inc.  
40                       Alumier Labs Inc.  
45                        Uniderm USA, Inc  
54                       ISDIN Corporation  
99                       NBTY Global, Inc.  
100                       Methapharm, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  Sep 26 2022 10:50AM  247255        10   
1  Sep 26 2022 10:46AM  247246        21   
2  Sep 26 2022 10:41AM  247253        15   
3  Sep 26 2022 10:40AM  247249        15   
4  Sep 26 2022 10:39AM  247248        15   
5  Sep 26 2022 10:29AM  247252        20   
6  Sep 26 2022 10:05AM  247247        12   
7  Sep 26 2022 10:03AM  247229        10   
8  Sep 26 2022  9:56AM  247199        21   
9  Sep 26 2022  9:54AM  247242        10   

                                Customer Executing Released  
0                       Eywa Pharma Inc.                  1  
1                      NBTY Global, Inc.                  1  
2  Carwin Pharmaceutical Associates, LLC                  6  
3        Brookfield Pharmaceuticals, LLC                 24  
4                  Alliance Pharma, Inc.                  8  
5                      Alumier Labs Inc.                  5  
6                       Uniderm USA, Inc         8        1  
7                      ISDIN Corporation                 45  
8                      NBTY Global, Inc.         1           
9                       Methapharm, Inc.                  6

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  Sep 26 2022 10:50AM  247255        10   
1  Sep 26 2022 10:46AM  247246        21   
2  Sep 26 2022 10:41AM  247253        15   
3  Sep 26 2022 10:40AM  247249        15   
4  Sep 26 2022 10:39AM  247248        15   
5  Sep 26 2022 10:29AM  247252        20   
6  Sep 26 2022 10:05AM  247247        12   
7  Sep 26 2022 10:03AM  247229        10   
8  Sep 26 2022  9:56AM  247199        21   
9  Sep 26 2022  9:54AM  247242        10   

                                Customer Released Executing  
0                       Eywa Pharma Inc.        1            
1                      NBTY Global, Inc.        1            
2  Carwin Pharmaceutical Associates, LLC        6            
3        Brookfield Pharmaceuticals, LLC       24            
4                  Alliance Pharma, Inc.        8            
5                      Alumier Labs Inc.        5            
6                       Uniderm USA, Inc        1         8  
7                      ISDIN Corporation       45            
8                      NBTY Global, Inc.                  1  
9                       Methapharm, Inc.        6

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  Sep 26 2022 10:50AM  247255        10   
1  Sep 26 2022 10:46AM  247246        21   
2  Sep 26 2022 10:41AM  247253        15   
3  Sep 26 2022 10:40AM  247249        15   
4  Sep 26 2022 10:39AM  247248        15   

                                Customer Released Executing  
0                       Eywa Pharma Inc.        1            
1                      NBTY Global, Inc.        1            
2  Carwin Pharmaceutical Associates, LLC        6            
3        Brookfield Pharmaceuticals, LLC       24            
4                  Alliance Pharma, Inc.        8

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse  \
0  Sep 26 2022 10:50AM  247255        10   
1  Sep 26 2022 10:46AM  247246        21   
2  Sep 26 2022 10:41AM  247253        15   
3  Sep 26 2022 10:40AM  247249        15   
4  Sep 26 2022 10:39AM  247248        15   

                                Customer  Released  Executing  
0                       Eywa Pharma Inc.       1.0        NaN  
1                      NBTY Global, Inc.       1.0        NaN  
2  Carwin Pharmaceutical Associates, LLC       6.0        NaN  
3        Brookfield Pharmaceuticals, LLC      24.0        NaN  
4                  Alliance Pharma, Inc.       8.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  Sep 26 2022 10:50AM  247255        10   
1  Sep 26 2022 10:46AM  247246        21   
2  Sep 26 2022 10:41AM  247253        15   
3  Sep 26 2022 10:40AM  247249        15   
4  Sep 26 2022 10:39AM  247248        15   

                                Customer  Released  Executing  
0                       Eywa Pharma Inc.       1.0        0.0  
1                      NBTY Global, Inc.       1.0        0.0  
2  Carwin Pharmaceutical Associates, LLC       6.0        0.0  
3        Brookfield Pharmaceuticals, LLC      24.0        0.0  
4                  Alliance Pharma, Inc.       8.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3461221     158.0       41.0
12          247247       1.0        8.0
15          741750      38.0        0.0
16          247238       1.0        0.0
20          247252       5.0        0.0
21          741642       1.0        2.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3461221     158.0       41.0
1        12   247247       1.0        8.0
2        15   741750      38.0        0.0
3        16   247238       1.0        0.0
4        20   247252       5.0        0.0
5        21   741642       1.0        2.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     158.0       41.0
1        12       1.0        8.0
2        15      38.0        0.0
3        16       1.0        0.0
4        20       5.0        0.0
5        21       1.0        2.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  158.0
1        12  Released    1.0
2        15  Released   38.0
3        16  Released    1.0
4        20  Released    5.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    15   16   20   21
Status                                    
Executing   41.0  8.0   0.0  0.0  0.0  2.0
Released   158.0  1.0  38.0  1.0  5.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    15   16   20   21
0          Executing   41.0  8.0   0.0  0.0  0.0  2.0
1           Released  158.0  1.0  38.0  1.0  5.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    15   16   20   21
0  Executing   41.0  8.0   0.0  0.0  0.0  2.0
1   Released  158.0  1.0  38.0  1.0  5.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()